In [9]:
import face_recognition
import cv2
import numpy as np
import csv
import os
from datetime import datetime
fr=face_recognition
from keras.models import load_model
from time import sleep
from tensorflow.keras.utils import load_img, img_to_array
import keras.utils as image

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
classifier =load_model('model.h5')

emotion_labels = ['Angry','Disgust','Fear','Happy','Neutral', 'Sad', 'Surprise']

In [10]:
video_capture=cv2.VideoCapture(0)

Shreya_image=face_recognition.load_image_file("faces/Shreya.jpeg")
Shreya_encoding=face_recognition.face_encodings(Shreya_image)[0]

APJ_image=face_recognition.load_image_file("faces/APJ Abdul kalam.jpg")
APJ_encoding=face_recognition.face_encodings(APJ_image)[0]

Nikola_image=face_recognition.load_image_file("faces/Nikola Tesla.jpeg")
Nikola_encoding=face_recognition.face_encodings(Nikola_image)[0]

Vishy_image=face_recognition.load_image_file("faces/Vishy.jpg")
Vishy_encoding=face_recognition.face_encodings(Vishy_image)[0]

In [11]:
known_face_encoding=[
    Shreya_encoding,
    APJ_encoding,
    Nikola_encoding,
    Vishy_encoding
]

known_faces_names=[
    "Shreya",
    "APJ Abdul Kalam",
    "Nikola Tesla",
    "Vishy"
]

In [12]:
students=known_faces_names.copy()

In [13]:
colours={"Angry": (0,0,255),"Disgust": (250,230,230),"Fear":(255,0,0),"Happy":(0,255,255),"Neutral":(0,255,0),"Sad":(255,255,255),"Surprise": (220,70,220)}

In [14]:
now=datetime.now()
current_date=now.strftime("%Y-%m-%d")

In [15]:
f=open(current_date+'.csv','w+',newline='')
lnwriter=csv.writer(f)

In [16]:
while True:
    
    ########################################
#     _,frame=video_capture.read()
#     small_frame=frame
#     rgb_small_frame=small_frame[:,:,::-1]
    
#     face_locations=face_recognition.face_locations(rgb_small_frame)
#     face_encodings=face_recognition.face_encodings(rgb_small_frame,face_locations)
#     face_names=[]
#     for face_encoding in face_encodings:
#         matches=face_recognition.compare_faces(known_face_encoding,face_encoding)
#         name=""
#         face_distance=face_recognition.face_distance(known_face_encoding,face_encoding)
#         best_match_index=np.argmin(face_distance)
#         if matches[best_match_index]:
#             name=known_faces_names[best_match_index]

#         face_names.append(name)
#         if name in known_faces_names:
#             if name in students:
#                 students.remove(name)
#                 print(students)
#                 current_time=now.strftime("%H-%M-%S")
#                 lnwriter.writerow([name,current_time])
    
    ############################################
    
    ret,frame=video_capture.read()

    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            label=emotion_labels[prediction.argmax()]

    rgb_frame=frame[:,:,::-1]
    fc_locations=fr.face_locations(rgb_frame)
    
    fc_encodings=fr.face_encodings(rgb_frame,fc_locations)
    
    for (top,right,bottom,left),face_encoding in zip(fc_locations,fc_encodings):
        matches=fr.compare_faces(known_face_encoding,face_encoding)
        
        name='Unknown'
        
        fc_distances=fr.face_distance(known_face_encoding,face_encoding)
        
        match_index=np.argmin(fc_distances)
        
        if matches[match_index]:
            name=known_faces_names[match_index]
        
        cv2.rectangle(frame,(left,top),(right,bottom),(0,0,0),2)
        
        cv2.rectangle(frame,(left,bottom),(right,bottom),(0,0,0),2)
        
        font=cv2.FONT_HERSHEY_SIMPLEX
        
        cv2.putText(frame,name,(left+6,bottom-6),font,1.0,(0,0,0),1)
        cv2.putText(frame,label,(left,top),font,1.0,colours[label],1)
        
        if name in students:
            students.remove(name)
            lnwriter.writerow([name])


        
    cv2.imshow("Attendance Frame",frame)

    if cv2.waitKey(1) & 0xFF == ord('q'): break

video_capture.release()
cv2.destroyAllWindows()
f.close()

1/1 [==============================] - 0s 27ms/step
